# Trabalho Final

## Coleta Preparação e Análise de Dados

#### Aléxia de Jesus Dorneles Pereira, Arthur Land Avila, Gustavo Giongo Lottermann, Matheus Gonzaga Krebs e Vicente Hofmeister

Trabalho feito utilizando [biblioteca Pysus](https://github.com/AlertaDengue/PySUS/tree/main). Caso necessário, confira a [documentação](https://pysus.readthedocs.io/en/latest/).

## Pysus

In [ ]:
import os
import pandas as pd
from pysus.online_data import SINAN as SINAN_Online
from pysus.online_data import SINASC as SINASC_Online
from pysus.online_data import SIM as SIM_ONLINE
from pysus.ftp.databases.sinan import SINAN
from pysus.ftp.databases.sinasc import SINASC
from pysus.ftp.databases.sim import SIM

Import de bibliotecas

A sessão de código a seguir salva em listas os anos que os dados de sinan, sinasc e sim estão disponiveis

In [ ]:
states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', \
          'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', \
          'RR', 'SC', 'SP', 'SE', 'TO']

sinan_years = SINAN_Online.get_available_years("DENG")
sinasc_years = SINASC_Online.get_available_years(group= 'DN', states = states)
sim_years = SIM_ONLINE.get_available_years(group= 'CID10', states = states)

Confere os anos em que todas bases de dados estão disponiveis simultaneamente

In [ ]:
avaliable_years = []

for year in sinan_years:
      if year in sinasc_years and year in sim_years:
            avaliable_years.append(year)

#### SINAN

busca arquivos SINAN baseado nos anos definidos em years, descreve-os e realiza download dos datasets

In [ ]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sinan = SINAN().load()
sinan_files = sinan.get_files(dis_code=['DENG'], year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sinan_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sinan.describe(file))
            sinan.download(file, local_dir=parquet_path)

#### SINASC

In [ ]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sinasc = SINASC().load()
sinasc_files = sinasc.get_files(group='DN', uf=states, year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sinasc_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sinasc.describe(file))
            sinasc.download(file, local_dir=parquet_path)

#### SIM

In [ ]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sim = SIM().load()
sim_files = sim.get_files('CID10', uf=states, year=years)

parquet_path = os.getcwd() + "/Data/Parquet/"
csv_path = os.getcwd() + '/Data/CSV/'

if not 'Parquet' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(parquet_path)

if not 'CSV' in os.listdir(os.getcwd() + "/Data") :
      os.mkdir(csv_path)

for file in sim_files:
      if not file.basename.replace('.dbc', '.parquet') in os.listdir(parquet_path) and not file.basename.replace('.dbc', '.csv') in os.listdir(csv_path):
            print(sim.describe(file))
            sim.download(file, local_dir=parquet_path)

#### Transformação Parquet para CSV

Transforma todos os .parquet em .csv

In [ ]:
error_log_path = os.getcwd() + '/Data/error_log.txt'

files_names = os.listdir(parquet_path)
error_files = []

for file in files_names:
    try:
        df = pd.read_parquet(path= parquet_path + file)
        print(file)
        df.to_csv(path_or_buf=(csv_path + file.replace('.parquet', '.csv')), sep=';')
        
        parquet_files = os.listdir(parquet_path + file)
        for pFile in parquet_files:
            os.remove(parquet_path + file + '/' + pFile)
        os.rmdir(parquet_path + file) 
    except Exception as e:
        error_files.append(file)
        print(f"Erro ao processar o arquivo: {file}")

        os.remove(path=parquet_path + file)



# Escrever os arquivos que causaram erro em um arquivo de texto
if len(error_files) > 0:
    with open(error_log_path, 'w') as f:
        for error_file in error_files:
            f.write(f"{error_file}\n")